# Loop 2 Analysis: Understanding the Gap to Target

## Key Questions:
1. Where is the gap concentrated (which N values)?
2. What would be needed per-N to reach the target?
3. What approaches haven't been tried yet?

In [1]:
import pandas as pd
import numpy as np
import json

# Load the current best valid submission
df = pd.read_csv('/home/submission/submission.csv')
print(f"Submission shape: {df.shape}")

# Parse the data
def parse_value(v):
    if isinstance(v, str) and v.startswith('s'):
        return float(v[1:])
    return float(v)

df['xf'] = df['x'].apply(parse_value)
df['yf'] = df['y'].apply(parse_value)
df['degf'] = df['deg'].apply(parse_value)
df['n_val'] = df['id'].apply(lambda x: int(str(x).split('_')[0]))

print(f"N values range: {df['n_val'].min()} to {df['n_val'].max()}")
print(f"Unique N values: {df['n_val'].nunique()}")

Submission shape: (20100, 4)
N values range: 1 to 200
Unique N values: 200


In [2]:
# Calculate score per N
def calculate_score_per_n(df):
    scores = []
    for n in range(1, 201):
        group = df[df['n_val'] == n]
        if len(group) == 0:
            continue
        x_min, x_max = group['xf'].min(), group['xf'].max()
        y_min, y_max = group['yf'].min(), group['yf'].max()
        # Need to account for tree polygon extent
        # Tree is roughly 0.7 wide and 1.0 tall
        # But rotation changes this - use approximate bounding
        side = max(x_max - x_min + 0.7, y_max - y_min + 1.0)
        score_n = (side ** 2) / n
        scores.append({'n': n, 'side': side, 'score': score_n, 'count': len(group)})
    return pd.DataFrame(scores)

scores_df = calculate_score_per_n(df)
print(f"Total score (approx): {scores_df['score'].sum():.6f}")
print(f"\nTop 10 N values by score contribution:")
print(scores_df.nlargest(10, 'score')[['n', 'side', 'score']])

Total score (approx): 86.879164

Top 10 N values by score contribution:
     n      side     score
1    2  1.522919  1.159640
0    1  1.000000  1.000000
3    4  1.864220  0.868829
4    5  1.963054  0.770716
2    3  1.494898  0.744906
6    7  2.222297  0.705515
5    6  2.014194  0.676163
7    8  2.253674  0.634881
8    9  2.329993  0.603207
11  12  2.689319  0.602703


In [3]:
# Target analysis
TARGET = 68.897509
CURRENT = 70.626088
GAP = CURRENT - TARGET

print(f"Current score: {CURRENT:.6f}")
print(f"Target score: {TARGET:.6f}")
print(f"Gap to close: {GAP:.6f} ({GAP/TARGET*100:.2f}%)")

# If we need to close 1.73 points across 200 N values
# Average improvement needed per N = 1.73/200 = 0.00865
print(f"\nAverage improvement needed per N: {GAP/200:.6f}")

# But score = S^2/n, so improvement is weighted by 1/n
# Small N values have much higher leverage

Current score: 70.626088
Target score: 68.897509
Gap to close: 1.728579 (2.51%)

Average improvement needed per N: 0.008643


In [4]:
# Analyze what side length reduction is needed per N to reach target
# Current: sum(S_n^2/n) = 70.626
# Target: sum(S_n^2/n) = 68.897
# Need to reduce by 1.729

# If we reduce ALL side lengths by factor k:
# New score = sum((k*S_n)^2/n) = k^2 * sum(S_n^2/n) = k^2 * 70.626
# For k^2 * 70.626 = 68.897
# k^2 = 68.897/70.626 = 0.9755
# k = 0.9877

k_needed = np.sqrt(TARGET / CURRENT)
print(f"If all side lengths reduced uniformly:")
print(f"  Reduction factor k = {k_needed:.6f}")
print(f"  Each side needs to be {(1-k_needed)*100:.2f}% smaller")
print(f"  For a side of 1.0, need to reduce by {(1-k_needed):.4f} units")

# This is a 1.23% reduction in all side lengths
# For N=1 with side ~0.813, need to reduce by 0.01 units
# For N=200 with side ~5.5, need to reduce by 0.068 units

If all side lengths reduced uniformly:
  Reduction factor k = 0.987687
  Each side needs to be 1.23% smaller
  For a side of 1.0, need to reduce by 0.0123 units


In [5]:
# Load snapshot experiment history to understand what's been tried
import os

snapshot_path = '/home/nonroot/snapshots/santa-2025/21222392487/code/session_state.json'
if os.path.exists(snapshot_path):
    with open(snapshot_path) as f:
        snapshot_state = json.load(f)
    
    print(f"Snapshot has {len(snapshot_state.get('experiments', []))} experiments")
    print(f"Snapshot has {len(snapshot_state.get('submissions', []))} submissions")
    
    # List experiment names and scores
    print("\nExperiment history:")
    for exp in snapshot_state.get('experiments', []):
        print(f"  {exp['name']}: {exp.get('cv_score', 'N/A'):.6f}")
else:
    print("Snapshot not found")

Snapshot has 38 experiments
Snapshot has 13 submissions

Experiment history:
  001_baseline: 70.647327
  002_ensemble: 70.647306
  003_validated_ensemble: 70.647327
  004_bbox3_optimization: 70.647326
  005_baseline_validated: 70.647327
  006_zaburo_grid: 70.647327
  007_sa_optimization: 70.647327
  008_repair_ensemble: 70.647327
  009_fractional_translation: 70.647327
  010_tessellation_and_ensemble: 70.630478
  011_random_restart_sa: 70.630478
  012_scanline_packer: 70.630478
  013_long_sa: 70.630478
  014_basin_hopping: 70.630478
  015_constraint_programming: 70.630478
  016_rebuild_corners: 70.630465
  017_cross_n_extraction: 70.630465
  018_egortrushin_tessellation: 70.630478
  019_cpp_sa: 70.630455
  020_asymmetric_solutions: 70.630455
  021_tessellation_search: 70.630429
  022_exhaustive_small_n: 70.627582
  023_invalid_snapshot_analysis: 70.627582
  024_exhaustive_n1: 70.627582
  025_iterative_refinement: 70.626088
  026_thorough_cascade: 70.624381
  027_iterative_sa_refinement

In [6]:
# Analyze submissions from snapshot
print("\nSubmission history from snapshot:")
for sub in snapshot_state.get('submissions', []):
    lb = sub.get('lb_score', 'N/A')
    cv = sub.get('cv_score', 'N/A')
    error = sub.get('error', '')
    print(f"  {sub.get('model_name', 'unknown')}: CV={cv}, LB={lb}, Error={error[:50] if error else 'None'}")

# Count valid vs invalid submissions
valid_subs = [s for s in snapshot_state.get('submissions', []) if not s.get('error')]
invalid_subs = [s for s in snapshot_state.get('submissions', []) if s.get('error')]
print(f"\nValid submissions: {len(valid_subs)}")
print(f"Invalid submissions (overlaps): {len(invalid_subs)}")


Submission history from snapshot:
  001_baseline: CV=70.647327, LB=70.647326897636, Error=None
  002_ensemble: CV=70.647306, LB=, Error=Overlapping trees in group 042
  003_validated_ensemble: CV=70.647327, LB=70.647326897636, Error=None
  004_bbox3_optimization: CV=70.647326, LB=, Error=Overlapping trees in group 016
  010_tessellation_and_ensemble: CV=70.630478, LB=70.630478453757, Error=None
  011_random_restart_sa: CV=70.630478, LB=70.630478453757, Error=None
  018_egortrushin_tessellation: CV=70.630478, LB=70.630465007968, Error=None
  019_cpp_sa: CV=70.630455, LB=70.630454595919, Error=None
  022_exhaustive_small_n: CV=70.627582, LB=70.627582179198, Error=None
  025_iterative_refinement: CV=70.626088, LB=70.626088313081, Error=None
  026_thorough_cascade: CV=70.624381, LB=, Error=Overlapping trees in group 040
  035_genetic_algorithm_medium_n: CV=70.624381, LB=, Error=Overlapping trees in group 040
  036_very_long_optimization: CV=70.626088, LB=70.626088313081, Error=None

Valid

In [7]:
# Key insight: What approaches HAVEN'T been tried?
# From the snapshot experiments:
# - bbox3 optimization ✓
# - SA optimization ✓
# - Tessellation ✓
# - Genetic algorithm ✓
# - MIP for small N ✓
# - Ensemble from multiple sources ✓
# - Asymmetric perturbations ✓
# - Deletion cascade ✓
# - Corner rebuild ✓

# What's NOT in the list:
# 1. Reinforcement Learning (mentioned in web research)
# 2. Mixed-Integer Programming for medium N (only small N tried)
# 3. Constraint Programming with global constraints
# 4. Lattice-based initialization with optimal spacing
# 5. Cross-pollination from top LB submissions (not public)

print("Approaches tried in snapshot:")
approaches_tried = [
    "bbox3 C++ optimizer",
    "Simulated Annealing (SA)",
    "Tessellation patterns",
    "Genetic Algorithm",
    "MIP for small N",
    "Ensemble from multiple sources",
    "Asymmetric perturbations",
    "Deletion cascade",
    "Corner rebuild",
    "Fractional translation",
    "Basin hopping",
    "Random restart SA",
    "Greedy beam search"
]
for a in approaches_tried:
    print(f"  ✓ {a}")

print("\nApproaches NOT tried:")
approaches_not_tried = [
    "Reinforcement Learning",
    "MIP for medium N (10-50)",
    "Constraint Programming with global constraints",
    "Lattice-based initialization with optimal spacing",
    "No-fit polygon (NFP) based placement",
    "Jostle algorithm",
    "Density gradient flow",
    "Global boundary tension optimization"
]
for a in approaches_not_tried:
    print(f"  ✗ {a}")

Approaches tried in snapshot:
  ✓ bbox3 C++ optimizer
  ✓ Simulated Annealing (SA)
  ✓ Tessellation patterns
  ✓ Genetic Algorithm
  ✓ MIP for small N
  ✓ Ensemble from multiple sources
  ✓ Asymmetric perturbations
  ✓ Deletion cascade
  ✓ Corner rebuild
  ✓ Fractional translation
  ✓ Basin hopping
  ✓ Random restart SA
  ✓ Greedy beam search

Approaches NOT tried:
  ✗ Reinforcement Learning
  ✗ MIP for medium N (10-50)
  ✗ Constraint Programming with global constraints
  ✗ Lattice-based initialization with optimal spacing
  ✗ No-fit polygon (NFP) based placement
  ✗ Jostle algorithm
  ✗ Density gradient flow
  ✗ Global boundary tension optimization


In [8]:
# Analyze the crystallization pattern from the Why Not kernel
# The kernel shows that trees form lattice patterns with alternating orientations

# Let's analyze the current solution's orientation distribution
print("Orientation analysis:")
for n in [10, 50, 100, 150, 200]:
    group = df[df['n_val'] == n]
    angles = group['degf'].values % 360
    
    # Count trees pointing up (0-90 or 270-360) vs down (90-270)
    up = np.sum((angles <= 90) | (angles >= 270))
    down = np.sum((angles > 90) & (angles < 270))
    
    print(f"  N={n}: Up={up}, Down={down}, Ratio={up/(up+down):.2f}")

# If the ratio is close to 0.5, trees are alternating orientations
# This is the "crystallization" pattern mentioned in the kernel

Orientation analysis:
  N=10: Up=8, Down=2, Ratio=0.80
  N=50: Up=26, Down=24, Ratio=0.52
  N=100: Up=50, Down=50, Ratio=0.50
  N=150: Up=78, Down=72, Ratio=0.52
  N=200: Up=101, Down=99, Ratio=0.51


In [9]:
# Check if there are any candidate files in the snapshot that might be better
import os

candidate_dir = '/home/nonroot/snapshots/santa-2025/21222392487/code/submission_candidates'
if os.path.exists(candidate_dir):
    candidates = os.listdir(candidate_dir)
    print(f"Found {len(candidates)} candidate files in snapshot")
    
    # Score a few candidates to see if any are better
    from shapely.geometry import Polygon
    from shapely import affinity
    from shapely.strtree import STRtree
    
    def get_tree_polygon(x, y, deg):
        TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
        TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
        p = Polygon(zip(TX, TY))
        p = affinity.rotate(p, deg, origin=(0, 0))
        p = affinity.translate(p, x, y)
        return p
    
    def score_submission(filepath):
        df = pd.read_csv(filepath)
        df['xf'] = df['x'].apply(lambda v: float(str(v).lstrip('s')))
        df['yf'] = df['y'].apply(lambda v: float(str(v).lstrip('s')))
        df['degf'] = df['deg'].apply(lambda v: float(str(v).lstrip('s')))
        df['n_val'] = df['id'].apply(lambda x: int(str(x).split('_')[0]))
        
        total_score = 0
        for n in range(1, 201):
            group = df[df['n_val'] == n]
            if len(group) != n:
                return None  # Invalid
            
            # Get bounding box of all tree polygons
            all_coords = []
            for _, row in group.iterrows():
                poly = get_tree_polygon(row['xf'], row['yf'], row['degf'])
                coords = np.array(poly.exterior.coords)
                all_coords.append(coords)
            all_coords = np.vstack(all_coords)
            
            x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
            y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
            side = max(x_range, y_range)
            total_score += (side ** 2) / n
        
        return total_score
    
    # Score the current submission
    current_score = score_submission('/home/submission/submission.csv')
    print(f"\nCurrent submission score: {current_score:.6f}")
else:
    print("Candidate directory not found")

Found 38 candidate files in snapshot



Current submission score: 70.626088


In [ ]:
# Summary of findings
print("="*60)
print("LOOP 2 ANALYSIS SUMMARY")
print("="*60)
print(f"\nCurrent best valid LB score: 70.626088")
print(f"Target score: 68.897509")
print(f"Gap: 1.728579 (2.51%)")
print(f"\nKey findings:")
print("1. 38 experiments in snapshot all converged to ~70.624")
print("2. All standard approaches (SA, GA, bbox3, MIP, ensemble) exhausted")
print("3. Need ~1.23% reduction in ALL side lengths to reach target")
print("4. Local overlap validation is LESS strict than Kaggle's")
print("5. Trees show crystallization pattern (alternating orientations)")
print(f"\nRecommended next steps:")
print("1. Try fundamentally different initialization (lattice-based)")
print("2. Use more conservative overlap detection (add buffer)")
print("3. Focus on N=2-50 where improvements have highest leverage")
print("4. Try NFP-based placement for guaranteed non-overlap")
print("5. Research what top LB teams are doing differently")